In [1]:
%matplotlib inline
from src.ptp.models.gan_3d.gan3d import Generator
from src.ptp.models.gan_3d.discriminator import Discriminator
from src.ptp.models.gan_3d.gan3d import GAN3D
from src.ptp.evaluation.visualization import visualize_slices, visualize_volumes
from pathlib import Path

import matplotlib.pyplot as plt
import nibabel as nib  
import numpy as np

from monai.data import MetaTensor
import pytorch_lightning as pl
import torch
from torch import nn
import os

In [3]:
DATA_DIR = Path('../data/generated_part1_nii_gz')
TARGET_DATA_DIR = DATA_DIR
INPUT_DIR = Path('../data')
MODELS_DIR = INPUT_DIR / 'models'
TRAINING_DIR = INPUT_DIR / 'training'
os.makedirs(TRAINING_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

In [4]:
percentile = 5
n_critic = 4

In [5]:
gan = GAN3D(target_data_dir=TARGET_DATA_DIR, percentile=percentile, n_critic=n_critic)

In [6]:
max_epochs = 10

checkpoint_dir = TRAINING_DIR / 'test-gan'

checkpoint_callback = pl.callbacks.ModelCheckpoint(dirpath=checkpoint_dir, filename='{epoch}-{val_loss:.2f}-{train_loss:.2f}', 
                                      monitor='val_loss', save_last=True) 
early_stopping = pl.callbacks.EarlyStopping(monitor='val_loss', verbose=True)

trainer = pl.Trainer(
    max_epochs=max_epochs,
    default_root_dir=checkpoint_dir,
    callbacks=[checkpoint_callback],
    fast_dev_run=True
)

trainer.fit(gan)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.

  | Name | Type          | Params | Mode 
-----------------------------------------------
0 | G    | Generator     | 587 K  | train
1 | D    | Discriminator | 410 K  | train
-----------------------------------------------
997 K     Trainable params
0         Non-trainable params
997 K     Total params
3.988     Total estimated model params size (MB)
Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.
Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Epoch 0: 100%|██████████| 1/1 [01:08<00:00,  0.01it/s, g_loss_step=2.170, d_loss_step=0.590, n_step_step=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 1/1 [01:40<00:00,  0.01it/s, g_loss_step=2.170, d_loss_step=0.590, n_step_step=0.000, val_g_loss=0.424, val_d_loss=0.658, val_loss=1.080, g_loss_epoch=2.170, d_loss_epoch=0.590, n_step_epoch=0.000]

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 0: 100%|██████████| 1/1 [01:40<00:00,  0.01it/s, g_loss_step=2.170, d_loss_step=0.590, n_step_step=0.000, val_g_loss=0.424, val_d_loss=0.658, val_loss=1.080, g_loss_epoch=2.170, d_loss_epoch=0.590, n_step_epoch=0.000]
